<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Resumen_Estadistico_Auscultacion_Presas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={'AS175_20250608.csv': {'metadata': {'name': 'AS175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Mostrando: PUNTOS_FIJOS_MD (CSV)


,FECHA,MARGEN,INSTRUMENTO,DELTA_NORTE_[M],DELTA_ESTE_[M],DELTA_COTA_[M],DISTANCIA_[M],DISTANCIA_(MM),AZIMUT_REF._AL_NORTE,TASA_NORTE_(MM/DIA),TASA_ESTE_(MM/DIA),TASA_COTA_(MM/DIA),TASA_DISTANCIA_(MM/DIA)
0,04/10/2021,Derecha,PF01 PRIMA,0.000333,-0.000667,0.003333,0.000745,0.745356,296.565051,0.041667,-0.083333,0.416667,0.093169
1,10/10/2021,Derecha,PF01 PRIMA,-0.000667,-0.000667,0.004333,0.000943,0.942809,225.000000,-0.047619,-0.047619,0.309524,0.067344
2,14/10/2021,Derecha,PF01 PRIMA,0.003333,0.003333,0.003333,0.004714,4.714045,45.000000,0.185185,0.185185,0.185185,0.261891
3,18/10/2021,Derecha,PF01 PRIMA,0.002333,0.002333,0.000333,0.003300,3.299832,45.000000,0.106061,0.106061,0.015152,0.149992
4,27/10/2021,Derecha,PF01 PRIMA,-0.000667,-0.000667,0.005333,0.000943,0.942809,225.000000,-0.021505,-0.021505,0.172043,0.030413


In [2]:
import pandas as pd

# Asume que datos_csv y datos_xlsx ya están definidos

# Diccionario de instrumentos, fecha, ID y variables numéricas (ajusta según tus datos)
instrumentos_info = {
    "Puntos Fijos MI": {"df": datos_csv["puntos_fijos_mi"], "fecha_col": "FECHA", "id_col": "INSTRUMENTO"},
    "Puntos Fijos MD": {"df": datos_csv["puntos_fijos_md"], "fecha_col": "FECHA", "id_col": "INSTRUMENTO"},
    "Piezómetros Eléctricos": {"df": datos_csv["piezometros_electricos"], "fecha_col": "FECHA", "id_col": "PIEZOMETRO"},
    "Piezómetros Casagrande": {"df": datos_csv["piezometros_casagrande"], "fecha_col": "FECHA", "id_col": "PIEZOMETRO"},
    "Inclinómetros": {"df": datos_csv["inclinometros"], "fecha_col": "Fecha", "id_col": "Inclinometro"},
    "Celdas de Asentamiento": {"df": datos_csv["asentamiento"], "fecha_col": "FECHA", "id_col": "CELDA_DE_ASENTAMIENTO"},
    "Freatímetros": {"df": datos_csv["freatimetros"], "fecha_col": "FECHA", "id_col": "FREATIMETRO"},
    "Extensómetros": {"df": datos_csv["extensometro"], "fecha_col": "FECHA", "id_col": "EXTENSOMETRO"}
}

# Recorrer y mostrar resumen estadístico
resumenes = {}

for nombre, info in instrumentos_info.items():
    df = info["df"].copy()
    if df.empty:
        print(f"\n[{nombre}] Sin datos.\n")
        continue
    # Convertir fecha y limpiar
    fecha_col = info["fecha_col"]
    if fecha_col in df.columns:
        df[fecha_col] = pd.to_datetime(df[fecha_col], dayfirst=True, errors='coerce')
    # Seleccionar columnas numéricas útiles
    num_cols = [col for col in df.select_dtypes(include='number').columns]
    # Incluir "Cota Nivel Freático" si existe como texto o COTA_NF
    for extra_col in ["Cota Nivel Freático", "COTA_NF"]:
        if extra_col in df.columns:
            num_cols.append(extra_col)
    num_cols = list(set(num_cols))
    if not num_cols:
        print(f"[{nombre}] Sin variables numéricas relevantes.\n")
        continue
    print(f"\n{'='*50}\nInstrumento: {nombre}\n")
    resumen = df[num_cols].describe(percentiles=[.25, .5, .75]).T
    resumen = resumen.rename(columns={"50%": "mediana", "std": "desv_std", "min": "mínimo", "max": "máximo", "count": "cantidad"})
    display(resumen[["cantidad", "mean", "mediana", "desv_std", "mínimo", "máximo"]].round(4))
    resumenes[nombre] = resumen

# (Opcional) Exportar todos los resúmenes a un solo Excel
with pd.ExcelWriter("resumen_estadistico_instrumentos.xlsx") as writer:
    for nombre, resumen in resumenes.items():
        resumen.to_excel(writer, sheet_name=nombre[:31])  # Sheet name max 31 chars
print("Archivo Excel generado: resumen_estadistico_instrumentos.xlsx")


Instrumento: Puntos Fijos MI



,cantidad,mean,mediana,desv_std,mínimo,máximo
TASA_ESTE_(MM/DIA),2727.0,0.0007,0.0000,0.0550,-1.0435,1.0714
TASA_DISTANCIA_(MM/DIA),2727.0,0.0281,0.0004,0.0747,-0.0324,1.0714
TASA_NORTE_(MM/DIA),2157.0,-0.0072,0.0000,0.0647,-0.5938,0.7619
AZIMUT_REF._AL_NORTE,2723.0,178.1805,180.0000,137.0568,0.0000,360.0000
DISTANCIA_(MM),2582.0,5.5032,1.0541,8.8498,-10.0000,148.1773
DELTA_COTA_[M],2727.0,0.0004,0.0000,0.0072,-0.0720,0.0600
DELTA_NORTE_[M],2157.0,-0.0030,0.0000,0.0082,-0.0839,0.1464
TASA_COTA_(MM/DIA),2727.0,0.0194,0.0000,0.1247,-0.5476,1.2143
DISTANCIA_[M],2727.0,0.0054,0.0010,0.0089,-0.0100,0.1482
DELTA_ESTE_[M],2727.0,-0.0002,0.0000,0.0069,-0.0710,0.0497



Instrumento: Puntos Fijos MD



,cantidad,mean,mediana,desv_std,mínimo,máximo
TASA_ESTE_(MM/DIA),2701.0,0.0008,0.0000,0.0558,-0.8750,0.7500
TASA_DISTANCIA_(MM/DIA),2701.0,0.0292,0.0007,0.0720,0.0000,0.9530
TASA_NORTE_(MM/DIA),2701.0,-0.0003,0.0002,0.0540,-0.6125,0.9286
AZIMUT_REF._AL_NORTE,2672.0,229.7453,256.7433,125.6512,0.0000,360.0000
DISTANCIA_(MM),2578.0,10.2326,6.7798,11.5435,0.0000,83.9107
DELTA_COTA_[M],2701.0,-0.0044,0.0000,0.0120,-0.0763,0.0283
DELTA_NORTE_[M],2701.0,0.0031,0.0001,0.0128,-0.0379,0.0801
TASA_COTA_(MM/DIA),2701.0,-0.0011,0.0000,0.0974,-0.9286,0.9286
DISTANCIA_[M],2701.0,0.0100,0.0065,0.0115,0.0000,0.0839
DELTA_ESTE_[M],2701.0,-0.0021,0.0000,0.0074,-0.0317,0.0540



Instrumento: Piezómetros Eléctricos



...


Instrumento: Freatímetros



,cantidad,mean,mediana,desv_std,mínimo,máximo
PROF_NIVEL_FREATICO_(M),199.0,4.8088,4.860,1.1428,2.730,6.910
COTA_NIVEL_FREATICO_(M.S.N.M.),199.0,115.2948,115.444,0.8842,113.873,116.701
COTA_RÌO_(M.S.N.M),180.0,115.9050,115.969,1.1325,114.069,117.972
PRECIPITACIONES,125.0,4.7680,1.000,7.2590,0.000,25.400
CARGA_(M.C.A),199.0,10.9284,10.910,1.1071,8.885,12.950



Instrumento: Extensómetros



,cantidad,mean,mediana,desv_std,mínimo,máximo
COTA_FINAL_EXCAV._(MSNM),197.0,98.7614,96.0,10.2445,87.5,120.0
PROFUNDIDAD,773.0,5.2625,4.0,5.7017,0.0,19.9
DIFERENCIAS_(MM),738.0,0.1978,0.0,1.0374,-2.0,7.0
Z/COTA_RELEV.,738.0,114.3182,116.0,5.8470,100.5,120.5
ACUMULADO_(MM),764.0,0.2945,0.0,1.2492,0.0,7.0
COTA_EXCAV._(MSNM),774.0,114.0468,116.0,7.7701,11.3,120.5


Archivo Excel generado: resumen_estadistico_instrumentos.xlsx


In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Asume que datos_csv ya están definidos

instrumentos_info = {
    "Puntos Fijos MI": {"df": datos_csv["puntos_fijos_mi"], "fecha_col": "FECHA", "id_col": "INSTRUMENTO"},
    "Puntos Fijos MD": {"df": datos_csv["puntos_fijos_md"], "fecha_col": "FECHA", "id_col": "INSTRUMENTO"},
    "Piezómetros Eléctricos": {"df": datos_csv["piezometros_electricos"], "fecha_col": "FECHA", "id_col": "PIEZOMETRO"},
    "Piezómetros Casagrande": {"df": datos_csv["piezometros_casagrande"], "fecha_col": "FECHA", "id_col": "PIEZOMETRO"},
    "Inclinómetros": {"df": datos_csv["inclinometros"], "fecha_col": "Fecha", "id_col": "Inclinometro"},
    "Celdas de Asentamiento": {"df": datos_csv["asentamiento"], "fecha_col": "FECHA", "id_col": "CELDA_DE_ASENTAMIENTO"},
    "Freatímetros": {"df": datos_csv["freatimetros"], "fecha_col": "FECHA", "id_col": "FREATIMETRO"},
    "Extensómetros": {"df": datos_csv["extensometro"], "fecha_col": "FECHA", "id_col": "EXTENSOMETRO"}
}

# Selector de instrumento
instrumento_sel = widgets.Dropdown(
    options=list(instrumentos_info.keys()),
    description="Instrumento:"
)

# Placeholder para otros selectores
variable_sel = widgets.Dropdown(description="Variable:")
id_sel = widgets.Dropdown(description="ID/Punto:")
anio_sel = widgets.Dropdown(description="Año:", options=["Todos"], value="Todos")

# Función para actualizar variables y puntos al cambiar instrumento
def actualizar_selectores(*args):
    info = instrumentos_info[instrumento_sel.value]
    df = info["df"]
    if df.empty:
        variable_sel.options = []
        id_sel.options = []
        anio_sel.options = ["Todos"]
        return
    # Variables numéricas
    num_cols = list(df.select_dtypes(include='number').columns)
    # Incluir Cota Nivel Freático si existe
    for extra_col in ["Cota Nivel Freático", "COTA_NF"]:
        if extra_col in df.columns:
            num_cols.append(extra_col)
    variable_sel.options = num_cols
    # IDs/puntos
    id_col = info["id_col"]
    if id_col in df.columns:
        ids = df[id_col].dropna().unique().tolist()
        id_sel.options = ["Todos"] + sorted(ids)
    else:
        id_sel.options = ["Todos"]
    # Años
    fecha_col = info["fecha_col"]
    if fecha_col in df.columns:
        df[fecha_col] = pd.to_datetime(df[fecha_col], errors='coerce')
        años = df[fecha_col].dt.year.dropna().unique()
        años = [str(int(a)) for a in años]
        anio_sel.options = ["Todos"] + sorted(años)
    else:
        anio_sel.options = ["Todos"]

instrumento_sel.observe(actualizar_selectores, names="value")
actualizar_selectores()

# Función de resumen estadístico
def mostrar_resumen(_=None):
    clear_output(wait=True)
    display(instrumento_sel, variable_sel, id_sel, anio_sel, boton)
    info = instrumentos_info[instrumento_sel.value]
    df = info["df"].copy()
    fecha_col = info["fecha_col"]
    id_col = info["id_col"]
    var = variable_sel.value
    if df.empty or var not in df.columns:
        print("No hay datos o variable seleccionada incorrecta.")
        return
    # Filtrar por ID/punto
    if id_sel.value != "Todos" and id_col in df.columns:
        df = df[df[id_col] == id_sel.value]
    # Filtrar por año
    if fecha_col in df.columns:
        df[fecha_col] = pd.to_datetime(df[fecha_col], errors='coerce')
        if anio_sel.value != "Todos":
            df = df[df[fecha_col].dt.year == int(anio_sel.value)]
    # Quitar nulos
    df = df[[fecha_col, var]].dropna()
    if df.empty:
        print("No hay datos para los filtros seleccionados.")
        return
    # Estadísticas
    cantidad = len(df)
    media = df[var].mean()
    mediana = df[var].median()
    desv_std = df[var].std()
    minimo = df[var].min()
    fecha_min = df.loc[df[var].idxmin(), fecha_col]
    maximo = df[var].max()
    fecha_max = df.loc[df[var].idxmax(), fecha_col]
    # Mostrar tabla resumen
    resumen = pd.DataFrame({
        "Cantidad": [cantidad],
        "Media": [media],
        "Mediana": [mediana],
        "Desv. estándar": [desv_std],
        "Mínimo": [minimo],
        "Fecha Mín.": [fecha_min],
        "Máximo": [maximo],
        "Fecha Máx.": [fecha_max]
    })
    display(resumen.round(4))
    # Opcional: mostrar cuartiles
    cuartiles = df[var].quantile([0.25, 0.75])
    print(f"1er cuartil (Q1): {cuartiles.loc[0.25]:.4f}")
    print(f"3er cuartil (Q3): {cuartiles.loc[0.75]:.4f}")

# Botón para ejecutar análisis
boton = widgets.Button(description="Mostrar Resumen")
boton.on_click(mostrar_resumen)

# Mostrar los selectores y botón
display(instrumento_sel, variable_sel, id_sel, anio_sel, boton)

Dropdown(description='Instrumento:', index=1, options=('Puntos Fijos MI', 'Puntos Fijos MD', 'Piezómetros Eléc…

Dropdown(description='Variable:', index=4, options=('DELTA_NORTE_[M]', 'DELTA_ESTE_[M]', 'DELTA_COTA_[M]', 'DI…

Dropdown(description='ID/Punto:', options=('Todos', 'IN-C1', 'IN1-A1', 'IN1-B1', 'IN2-A1 ', 'IN2-B1', 'IN2-C1a…

Dropdown(description='Año:', options=('Todos', '2021', '2022', '2023', '2025', '2026'), value='Todos')

Button(description='Mostrar Resumen', style=ButtonStyle())

,Cantidad,Media,Mediana,Desv. estándar,Mínimo,Fecha Mín.,Máximo,Fecha Máx.
0,984,8.7789,5.8915,9.6019,0.0,2023-04-10,39.1511,2023-12-07


1er cuartil (Q1): 0.1000
3er cuartil (Q3): 14.5436


In [17]:
import pandas as pd
import numpy as np
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# ========== SELECTORES ==========
instrumentos = [
    "puntos_fijos_mi", "puntos_fijos_md", "inclinometros", "asentamiento",
    "piezometros_electricos", "piezometros_casagrande", "freatimetros", "extensometro"
]

instrumento_dropdown = widgets.Dropdown(
    options=[
        "Puntos Fijos", "Piezómetros Eléctricos", "Piezómetros Casagrande", "Inclinómetros",
        "Celdas de Asentamiento", "Freatímetros", "Extensómetros"
    ],
    value="Puntos Fijos",
    description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
margen_dropdown = widgets.Dropdown(description="Margen:", options=[])
punto_dropdown = widgets.Dropdown(description="Punto Fijo:", options=[])
variable_pf_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_pf_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
progresiva_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_pe_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_pe_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
margen_cg_dropdown = widgets.Dropdown(description="Margen:", options=[])
pz_cg_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_cg_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_cg_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
inclinometro_dropdown = widgets.Dropdown(description="Inclinómetro:", options=[])
anio_inc_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
eje_dropdown = widgets.Dropdown(options=["A+", "A-", "B+", "B-"], value="A+", description="Eje:")
progresiva_ca_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
celda_dropdown = widgets.Dropdown(description="Celda:", options=[])
variable_ca_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ca_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
freatimetro_dropdown = widgets.Dropdown(description="Freatímetro:", options=[])
variable_fr_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_fr_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
extensometro_dropdown = widgets.Dropdown(description="Extensómetro:", options=[])
variable_ex_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ex_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])

boton_estadisticas = widgets.Button(description="Calcular Estadísticas", button_style="primary")
output = widgets.Output()

# === FUNCIONES DE ACTUALIZACIÓN DE SELECTORES (idéntico a tus scripts) ===
def actualizar_opciones_pf(change=None):
    origen = origen_dropdown.value
    df_mi = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
    df_md = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
    datasets = {"Margen Izquierda (MI)": df_mi, "Margen Derecha (MD)": df_md}
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets:
        margen_dropdown.options = []
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = ["Todos"]
        return
    margen_dropdown.options = list(datasets.keys())
    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    if 'FECHA' not in df.columns:
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = ["Todos"]
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    variable_pf_dropdown.options = [col for col in df.select_dtypes(include='number').columns if col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique().tolist())
    anio_pf_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_opciones_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'FECHA' not in df.columns:
        progresiva_dropdown.options = []
        piezometro_dropdown.options = []
        variable_pe_dropdown.options = []
        anio_pe_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique().tolist())
    if progresiva_dropdown.options:
        progresiva_dropdown.value = progresiva_dropdown.options[0]
    actualizar_piezometros_pe()
    variable_pe_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']]
    anio_pe_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'PIEZOMETRO' not in df.columns:
        piezometro_dropdown.options = []
        return
    piezos = sorted(df[df['PROGRESIVA'] == progresiva_dropdown.value]['PIEZOMETRO'].dropna().unique().tolist())
    piezometro_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    if df.empty or 'MARGEN' not in df.columns or 'FECHA' not in df.columns:
        margen_cg_dropdown.options = []
        pz_cg_dropdown.options = []
        variable_cg_dropdown.options = []
        anio_cg_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    margen_cg_dropdown.options = sorted(df['MARGEN'].dropna().unique().tolist())
    if margen_cg_dropdown.options:
        margen_cg_dropdown.value = margen_cg_dropdown.options[0]
    actualizar_piezometros_cg()
    variable_cg_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'MARGEN', 'PIEZOMETRO']]
    anio_cg_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    if df.empty or 'MARGEN' not in df.columns or 'PIEZOMETRO' not in df.columns:
        pz_cg_dropdown.options = []
        return
    piezos = sorted(df[df['MARGEN'] == margen_cg_dropdown.value]['PIEZOMETRO'].dropna().unique().tolist())
    pz_cg_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_inc(change=None):
    df = datos_csv["inclinometros"] if origen_dropdown.value == "CSV" else datos_xlsx["inclinometros"]
    if df.empty or 'Inclinometro' not in df.columns or 'Fecha' not in df.columns:
        inclinometro_dropdown.options = []
        anio_inc_dropdown.options = ["Todos"]
        eje_dropdown.options = ["A+", "A-", "B+", "B-"]
        return
    df = df.copy()
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    inclinometro_dropdown.options = ["Todos"] + sorted(df['Inclinometro'].dropna().unique().tolist())
    anio_inc_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['Fecha'].dt.year.dropna().unique())]
    eje_dropdown.options = ["A+", "A-", "B+", "B-"]

def actualizar_opciones_ca(change=None):
    df = datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'FECHA' not in df.columns:
        progresiva_ca_dropdown.options = []
        celda_dropdown.options = []
        variable_ca_dropdown.options = []
        anio_ca_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_ca_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique().tolist())
    if progresiva_ca_dropdown.options:
        progresiva_ca_dropdown.value = progresiva_ca_dropdown.options[0]
    actualizar_celdas_ca()
    variable_ca_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'CELDA_DE_ASENTAMIENTO']]
    anio_ca_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_celdas_ca(change=None):
    df = datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'CELDA_DE_ASENTAMIENTO' not in df.columns:
        celda_dropdown.options = []
        return
    celdas = sorted(df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]['CELDA_DE_ASENTAMIENTO'].dropna().unique().tolist())
    celda_dropdown.options = ["Todas"] + list(celdas)

def actualizar_opciones_fr(change=None):
    df = datos_csv["freatimetros"] if origen_dropdown.value == "CSV" else datos_xlsx["freatimetros"]
    if df.empty or 'FREATIMETRO' not in df.columns or 'FECHA' not in df.columns:
        freatimetro_dropdown.options = []
        variable_fr_dropdown.options = []
        anio_fr_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    freatimetro_dropdown.options = ["Todos"] + sorted(df['FREATIMETRO'].dropna().unique().tolist())
    variable_fr_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'FREATIMETRO']]
    anio_fr_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_opciones_ex(change=None):
    df = datos_csv["extensometro"] if origen_dropdown.value == "CSV" else datos_xlsx["extensometro"]
    if df.empty or 'EXTENSOMETRO' not in df.columns or 'FECHA' not in df.columns:
        extensometro_dropdown.options = []
        variable_ex_dropdown.options = []
        anio_ex_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    extensometro_dropdown.options = ["Todos"] + sorted(df['EXTENSOMETRO'].dropna().unique().tolist())
    variable_ex_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'EXTENSOMETRO']]
    anio_ex_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value
    for w in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
              progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, anio_pe_dropdown,
              margen_cg_dropdown, pz_cg_dropdown, variable_cg_dropdown, anio_cg_dropdown,
              inclinometro_dropdown, anio_inc_dropdown, eje_dropdown,
              progresiva_ca_dropdown, celda_dropdown, variable_ca_dropdown, anio_ca_dropdown,
              freatimetro_dropdown, variable_fr_dropdown, anio_fr_dropdown,
              extensometro_dropdown, variable_ex_dropdown, anio_ex_dropdown]:
        w.layout.display = 'none'
    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    elif tipo == "Piezómetros Eléctricos":
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()
    elif tipo == "Piezómetros Casagrande":
        margen_cg_dropdown.layout.display = 'flex'
        pz_cg_dropdown.layout.display = 'flex'
        variable_cg_dropdown.layout.display = 'flex'
        anio_cg_dropdown.layout.display = 'flex'
        actualizar_opciones_cg()
    elif tipo == "Inclinómetros":
        inclinometro_dropdown.layout.display = 'flex'
        anio_inc_dropdown.layout.display = 'flex'
        eje_dropdown.layout.display = 'flex'
        actualizar_opciones_inc()
    elif tipo == "Celdas de Asentamiento":
        progresiva_ca_dropdown.layout.display = 'flex'
        celda_dropdown.layout.display = 'flex'
        variable_ca_dropdown.layout.display = 'flex'
        anio_ca_dropdown.layout.display = 'flex'
        actualizar_opciones_ca()
    elif tipo == "Freatímetros":
        freatimetro_dropdown.layout.display = 'flex'
        variable_fr_dropdown.layout.display = 'flex'
        anio_fr_dropdown.layout.display = 'flex'
        actualizar_opciones_fr()
    elif tipo == "Extensómetros":
        extensometro_dropdown.layout.display = 'flex'
        variable_ex_dropdown.layout.display = 'flex'
        anio_ex_dropdown.layout.display = 'flex'
        actualizar_opciones_ex()

# ====== FUNCIÓN DE RESUMEN ESTADÍSTICO (usa los filtros activos y .copy()) ======
def calcular_estadisticas(b=None):
    with output:
        clear_output(wait=True)
        tipo = instrumento_dropdown.value
        origen = origen_dropdown.value
        fecha_col = 'FECHA'
        variable = None
        df = None
        titulo = ""

        if tipo == "Puntos Fijos":
            df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
            if margen_dropdown.value == "Margen Derecha (MD)":
                df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
            variable = variable_pf_dropdown.value
            df = df.copy()
            if anio_pf_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)].copy()
            if punto_dropdown.value != "Todos":
                df = df[df['INSTRUMENTO'] == punto_dropdown.value].copy()
            titulo = f"{margen_dropdown.value}: {variable}"
        elif tipo == "Piezómetros Eléctricos":
            df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
            variable = variable_pe_dropdown.value
            df = df.copy()
            if progresiva_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_dropdown.value].copy()
            if piezometro_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value].copy()
            if anio_pe_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)].copy()
            titulo = f"{progresiva_dropdown.value} – {variable}"
        elif tipo == "Piezómetros Casagrande":
            df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
            variable = variable_cg_dropdown.value
            df = df.copy()
            if margen_cg_dropdown.value != "Todos":
                df = df[df['MARGEN'] == margen_cg_dropdown.value].copy()
            if pz_cg_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value].copy()
            if anio_cg_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)].copy()
            titulo = f"{margen_cg_dropdown.value} – {variable}"
        elif tipo == "Inclinómetros":
            df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
            fecha_col = 'Fecha'
            variable = eje_dropdown.value
            df = df.copy()
            if inclinometro_dropdown.value != "Todos":
                df = df[df['Inclinometro'] == inclinometro_dropdown.value].copy()
            if anio_inc_dropdown.value != "Todos":
                df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
                df = df[df['Fecha'].dt.year == int(anio_inc_dropdown.value)].copy()
            titulo = f"{inclinometro_dropdown.value} – {variable}"
        elif tipo == "Celdas de Asentamiento":
            df = datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
            variable = variable_ca_dropdown.value
            df = df.copy()
            if progresiva_ca_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value].copy()
            if celda_dropdown.value != "Todas":
                df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value].copy()
            if anio_ca_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)].copy()
            titulo = f"{progresiva_ca_dropdown.value} – {variable}"
        elif tipo == "Freatímetros":
            df = datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
            variable = variable_fr_dropdown.value
            df = df.copy()
            if freatimetro_dropdown.value != "Todos":
                df = df[df['FREATIMETRO'] == freatimetro_dropdown.value].copy()
            if anio_fr_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)].copy()
            titulo = f"{freatimetro_dropdown.value} – {variable}"
        elif tipo == "Extensómetros":
            df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
            variable = variable_ex_dropdown.value
            df = df.copy()
            if extensometro_dropdown.value != "Todos":
                df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value].copy()
            if anio_ex_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)].copy()
            titulo = f"{extensometro_dropdown.value} – {variable}"
        else:
            print("⚠️ Instrumento no reconocido."); return

        if variable not in df.columns:
            print(f"⚠️ La variable {variable} no está disponible en los datos."); return

        df[variable] = pd.to_numeric(df[variable], errors='coerce')
        df[fecha_col] = pd.to_datetime(df[fecha_col], dayfirst=True, errors='coerce')
        df = df.dropna(subset=[variable, fecha_col])
        if df.empty:
            print(f"⚠️ No hay datos válidos para la variable {variable} después de filtrar."); return

        datos = df[variable]
        conteo = datos.count()
        media = datos.mean()
        mediana = datos.median()
        moda = stats.mode(datos, keepdims=True)[0][0] if len(datos) > 0 else np.nan
        varianza = datos.var()
        desv_est = datos.std()
        minimo = datos.min()
        maximo = datos.max()
        idx_min = datos.idxmin()
        fecha_min = df.loc[idx_min, fecha_col] if idx_min in df.index else np.nan
        idx_max = datos.idxmax()
        fecha_max = df.loc[idx_max, fecha_col] if idx_max in df.index else np.nan
        q1 = datos.quantile(0.25)
        q3 = datos.quantile(0.75)

        print(f"📊 Estadísticas Descriptivas para {titulo}\n")
        print(f"Cantidad de datos: {conteo}")
        print(f"Media: {media:.4f}")
        print(f"Mediana: {mediana:.4f}")
        print(f"Moda: {moda:.4f}")
        print(f"Varianza: {varianza:.4f}")
        print(f"Desviación estándar: {desv_est:.4f}")
        print(f"Mínimo: {minimo:.4f} (Fecha: {fecha_min.date() if pd.notnull(fecha_min) else 'N/A'})")
        print(f"Máximo: {maximo:.4f} (Fecha: {fecha_max.date() if pd.notnull(fecha_max) else 'N/A'})")
        print(f"Primer cuartil (Q1): {q1:.4f}")
        print(f"Tercer cuartil (Q3): {q3:.4f}")

# ========== CONEXIONES Y DISPLAY ==========
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
margen_dropdown.observe(actualizar_opciones_pf, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')
margen_cg_dropdown.observe(actualizar_piezometros_cg, names='value')
progresiva_ca_dropdown.observe(actualizar_celdas_ca, names='value')
boton_estadisticas.on_click(calcular_estadisticas)

display(HTML("<h3 style='color:#1866a3'>Resumen Estadístico de Instrumentos de Auscultación</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown]))
display(widgets.HBox([
    margen_dropdown, punto_dropdown,
    progresiva_dropdown, piezometro_dropdown,
    margen_cg_dropdown, pz_cg_dropdown,
    inclinometro_dropdown,
    progresiva_ca_dropdown, celda_dropdown,
    freatimetro_dropdown,
    extensometro_dropdown
]))
display(widgets.HBox([
    variable_pf_dropdown, variable_pe_dropdown, variable_cg_dropdown,
    anio_pf_dropdown, anio_pe_dropdown, anio_cg_dropdown,
    anio_inc_dropdown, eje_dropdown,
    variable_ca_dropdown, anio_ca_dropdown,
    variable_fr_dropdown, anio_fr_dropdown,
    variable_ex_dropdown, anio_ex_dropdown
]))
display(boton_estadisticas)
display(output)

# Inicializar controles
actualizar_controles_visibles()

Button(button_style='primary', description='Calcular Estadísticas', style=ButtonStyle())

Output()